
# Аналитика кредитных заявок  
## Выявление минимального набора признаков для получения кредита


In [11]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', 100)



## 1. Загрузка и объединение данных

Даны две таблицы:
- `application_info.csv` — характеристики заявок,
- `default_flg.csv` — информация о дефолте.

Таблицы объединяются по идентификатору заявки `id`.


In [10]:

app = pd.read_csv('application_info.csv')
dfl = pd.read_csv('default_flg.csv')

df = app.merge(dfl, on='id')

df.head()


,id,application_dt,sample_cd,education_cd,gender_cd,age,car_own_flg,car_type_flg,appl_rej_cnt,good_work_flg,Score_bki,out_request_cnt,region_rating,home_address_cd,work_address_cd,income,SNA,first_time_cd,Air_flg,default_flg
0,1,01JAN2014,train,SCH,M,27,Y,Y,0,0,-1.917831,0,40,2,3,32000,1,4,N,0
1,2,01JAN2014,train,GRD,F,26,N,N,0,0,-1.153144,2,60,2,3,50000,2,1,N,0
2,3,01JAN2014,train,SCH,M,35,N,N,0,1,-1.732810,0,40,1,2,20000,4,3,N,0
3,4,01JAN2014,train,GRD,F,35,N,N,0,1,-2.552133,2,20,2,3,80000,1,3,N,0
4,5,01JAN2014,train,UGR,F,24,N,N,0,0,-1.914581,1,50,2,3,27000,1,2,N,0



## 2. Очистка данных

- пропуски удаляются,
- используется только выборка с известным значением `default_flg`.


In [3]:

df = df.dropna().copy()
df['default_flg'] = df['default_flg'].astype(int)

print('Размер выборки:', df.shape)
print('Средний дефолт-рейт:', round(df["default_flg"].mean(), 4))


Размер выборки: (180093, 20)
Средний дефолт-рейт: 0.1153



## 3. Базовый уровень риска

Рассчитаем общий дефолт-рейт — он будет использоваться как точка сравнения 
для отдельных признаков.


In [4]:

base_dr = df['default_flg'].mean()
base_dr


np.float64(0.11525156446946855)


## 4. Анализ отдельных признаков

Для каждого признака считаем дефолт-рейт по значениям и оцениваем:
- есть ли выраженная разница в риске,
- можно ли использовать признак как простой фильтр.


In [5]:

def simple_analysis(col):
    tab = (
        df.groupby(col)['default_flg']
        .agg(['count','mean'])
        .rename(columns={'mean':'default_rate'})
        .sort_values('default_rate')
    )
    return tab

display(simple_analysis('appl_rej_cnt'))
display(simple_analysis('out_request_cnt').head(10))
display(simple_analysis('region_rating'))


,count,default_rate
appl_rej_cnt,,
33,1,0.000000
24,1,0.000000
21,1,0.000000
18,2,0.000000
16,3,0.000000
13,5,0.000000
0,147984,0.099004
12,8,0.125000
1,21833,0.170384


,count,default_rate
out_request_cnt,,
53,1,0.000000
36,1,0.000000
34,1,0.000000
26,5,0.000000
22,8,0.000000
0,43239,0.088624
1,43752,0.108841
2,35248,0.120603
3,24087,0.126458


,count,default_rate
region_rating,,
80,29325,0.066155
70,15725,0.085024
60,39095,0.113979
50,66395,0.125341
40,28292,0.157536
30,689,0.175617
20,572,0.215035



### Комментарий

- Наличие хотя бы одного отказа (`appl_rej_cnt > 0`) резко увеличивает риск дефолта.
- Большое число внешних запросов также связано с повышенным риском.
- Региональный рейтинг демонстрирует монотонную связь с дефолтом.


In [6]:

# Анализ Score_bki через корзины
df['score_bin'] = pd.qcut(df['Score_bki'], 5, duplicates='drop')

df.groupby('score_bin')['default_flg'].agg(['count','mean'])


/var/folders/5y/6y1t278j0sxbw6bmg9dv31xh0000gn/T/ipykernel_48792/2491334875.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby('score_bin')['default_flg'].agg(['count','mean'])


,count,mean
score_bin,,
"(-3.626, -2.373]",36019,0.053000
"(-2.373, -2.099]",36018,0.078766
"(-2.099, -1.837]",36019,0.102779
"(-1.837, -1.532]",36018,0.138292
"(-1.532, 0.2]",36019,0.203420



### Комментарий

Кредитный скор (`Score_bki`) является наиболее информативным признаком:
при ухудшении значения дефолт-рейт возрастает монотонно.



## 5. Выбор минимального набора признаков

На основе анализа отдельных признаков отбираем **минимальный набор**, который:
- сильно влияет на риск,
- легко интерпретируется,
- не требует сложных расчётов.

Выбранные признаки:
1. `Score_bki` — основной индикатор кредитного риска,
2. `appl_rej_cnt` — история отказов,
3. `out_request_cnt` — кредитная активность,
4. `region_rating` — региональный фактор.


In [7]:

rule = (
    (df['Score_bki'] <= -1.7) &
    (df['appl_rej_cnt'] == 0) &
    (df['out_request_cnt'] <= 3) &
    (df['region_rating'] >= 50)
)

approved = df[rule]

print('Доля заявок, проходящих фильтр:', round(len(approved)/len(df), 3))
print('Дефолт-рейт в отобранной группе:', round(approved["default_flg"].mean(), 3))


Доля заявок, проходящих фильтр: 0.427
Дефолт-рейт в отобранной группе: 0.067



## 6. Итоговые выводы

1. Для базовой фильтрации заявок достаточно **небольшого числа признаков**.
2. Наиболее значимый фактор — кредитный скор (`Score_bki`).